# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Aprendizaje por refuerzos

### Trabajo práctico entregable 2/2 (materia completa)

**Estudiante:**
- [Chevallier-Boutell, Ignacio José.](https://www.linkedin.com/in/nachocheva/)

**Docentes:**
- Palombarini, Jorge (Mercado Libre).
- Barsce, Juan Cruz (Mercado Libre).

---

## Librerías

In [6]:
from typing import Callable, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
from gymnasium import spaces

import time

Para ver los videos de las ejecuciones hay que tener instalado ffmpeg (`apt-get install ffmpeg`).

---
# Ejercicio 1

Crear un entorno propio y entrenar agentes de RL en el mismo, utilizando diferentes algoritmos.

## Cheva's Odyssey: reglas del juego

El mapa del juego consiste en una grilla 6x6, como se muestra a continuación. Al comenzar un episodio, el agente se ubica en la posición $S$ (elemento [5, 5]) y debe llegar hasta $G$ (elemento [0, 2]) para terminar dicho episodio. El agente debe realizar tantos movimientos como sean necesarios para llegar a la meta y finalizar el episodio.

|J|A|G|A|~|~|
|-|-|-|-|-|-|
|~|A|A|A|~|~|
|~|~|~|~|~|~|
|~|~|~|~|~|~|
|A|A|~|~|~|~|
|P|A|~|~|~|S|

El espacio de estados $\mathcal{S}$ tiene 36 elementos. Para calcular el valor del estado asociado a cada elemento del mapa debemos calcular
    $$\text{Fila Actual} \times \text{Número de Columnas} + \text{Columna Actual}$$

donde debemos contar desde 0. De esta manera, el estado inicial $S$ es el estado 35 y el estado terminal $G$ es el estado 2.

El espacio de acciones $\mathcal{A}$ tiene 4 elementos para todos los estados que no están en el borde del mapa:
- 0 $\Rightarrow$ Se mueve hacia arriba.
- 1 $\Rightarrow$ Se mueve hacia la derecha.
- 2 $\Rightarrow$ Se mueve hacia abajo.
- 3 $\Rightarrow$ Se mueve hacia la izquierda.

Para los estados que están en los bordes, sólo se puede elegir entre 2 acciones si están en los vértices o entre 3 si están en las aristas, según corresponda. Observamos que se cumple que $\mathcal{A}, \mathcal{S} \in \mathbb{N}$.

Además de la salida $S$ y la meta $G$, tenemos otros elementos en el mapa. Los elementos vacíos (~) representan pasto, mientras que los elementos con $A$ son agua. El elemento $J$ es un jetpack y el elemento $P$ representa un premio extra. A partir de esto, la función de recompensa es tal que el agente recibe:
- $-1$ cuando ingresa en a un elemento con pasto o cuando busca el jetpack o el premio extra. Sus efectos no se pierden, pero tampoco se acumulan.
- $-8$ cuando ingresa en a un elemento con agua. En caso de contar con el jetpack, el costo por pasar por el agua se reduce a $-2$.
- $+0$ si alcanza la meta sin el premio extra y $+24$ cuando la alcanza con el premio extra.

Hay principalmente 5 caminos relevantes:
- **SG:** Ir directo a la meta requiere 8 pasos temporales, otorgando -14 puntos.
- **SJG:** Buscar el jetpack e ir a la meta requiere 12 pasos temporales, otorgando -12 puntos.
- **SPG:** Buscar el premio extra e ir a la meta requiere 12 pasos temporales, otorgando -8 puntos.
- **SPJG:** Buscar el premio extra, luego el jetpack e ir a la meta requiere 12 pasos temporales, otorgando -2 puntos.
- **SJPG:** Buscar el jetpack, luego el premio extra e ir a la meta requiere 22 pasos temporales, otorgando 0 puntos (puntuación máxima).

## Creación del agente

In [ ]:
class ChevasOdyssey(gym.Env):
    # Tipo de renderizado posible, además de None.
    metadata = {"render.modes": ["console"]}

    # Definimos los valores de las acciones
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3

    def __init__(self):
        super(ChevasOdyssey, self).__init__()

        # Tamaño del mapa
        self.grid_shape = (6, 6)

        # Inicializamos en agente en el punto de partida
        self.agent_pos = 35

        # Espacio de acciones
        self.action_space = spaces.Discrete(4)

        # Espacio de estados
        self.state_space = spaces.Discrete(36)


    def reset(self, seed=None) -> Tuple[np.array, dict]:
        """
        Reinicia el ambiente y devuelve la observación inicial
        """
        # Inicializamos en agente en el punto de partida
        self.agent_pos = 35

        # convertimos con astype a float32 (numpy) para hacer más general
        # el agente (en caso de que querramos usar acciones continuas)
        return (np.array([self.agent_pos]).astype(int), {})

    def step(self, action):
        if action == self.UP:
            self.agent_pos -= 1
        elif action == self.RIGHT:
            self.agent_pos += 1
        elif action == self.DOWN:
            self.agent_pos += 1
        elif action == self.LEFT:
            self.agent_pos += 1
        else:
            raise ValueError(
                f"Se recibió una acción inválida={action} que no es parte del\
                    espacio de acciones"
            )

        # Evitamos que el agente se salga de los límites de la grilla
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

        # Llegó el agente a su estado objetivo (izquierda) de la grilla?
        terminated = bool(self.agent_pos == 0)
        truncated = False  # no limitamos la duración de los episodios

        # Asignamos recompensa sólo cuando el agente llega a su objetivo
        # (recompensa = 0 en todos los demás estados)
        reward = 1 if self.agent_pos == 0 else 0

        # gym también nos permite devolver información adicional, ej. en Atari:
        # las vidas restantes del agente (no usaremos esto por ahora)
        info = {}

        return (
            np.array([self.agent_pos]).astype(np.float32),
            reward,
            terminated,
            truncated,
            info,
        )

    def render(self, mode="console"):
        if mode != "console":
            raise NotImplementedError()
        # en nuestra interfaz de consola, representamos el agente como una
        # cruz, y el resto como un punto
        print("." * self.agent_pos, end="")
        print("x", end="")
        print("." * (self.grid_size - self.agent_pos))

    def close(self):
        pass

In [1]:
nS = np.prod((6,6))
nS

36

In [4]:
isd = np.zeros(nS)
isd

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [5]:
isd[np.ravel_multi_index((3,0), (6,6))] = 1.0
isd

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])